In [1]:
import pandas as pd
import plotly.express as px

# Read Line 1 & Line 3 Transit time data

In [3]:
df = pd.read_excel(r"C:\Users\siwz@equinor.com\OneDrive - Equinor\personal\projects\Colonial Pipeline Forecast\colonial_transit_time.xlsx", sheet_name='line13')


# Read pipeline EIA data (PADD 3 to PADD 1)

In [4]:
EIA_df = pd.read_csv(r'C:\Users\siwz@equinor.com\OneDrive - Equinor\personal\projects\Colonial Pipeline Forecast\gasoline_pipeline_EIA.csv')
EIA_df

,period,value
0,2018-01,1528.900000
1,2018-02,1422.333333
2,2018-03,1501.033333
3,2018-04,1459.100000
4,2018-05,1566.933333
...,...,...
83,2024-12,1580.500000
84,2025-01,1520.400000
85,2025-02,1563.100000
86,2025-03,1558.333333


# Line 1 & 3 transit time further cleaning 

# We only have one transit time for each cycle, so just 6 cycle as a month

In [5]:
def monthly_avg_from_cycles(df):
    """
    Map cycles (1..72) to months (1..12, 6 cycles each) and
    compute the monthly average for each year column.

    df: DataFrame with a 'cycle' column and one column per year (e.g., 2020..2025)
    """
    year_cols = [c for c in df.columns if c != 'cycle']
    out = (
        df.assign(Month=((df['cycle'].astype(int) - 1) // 6 + 1))
          .groupby("Month", as_index=False)[year_cols].mean()
    )
    return out

# Example:
monthly_df = monthly_avg_from_cycles(df)
monthly_df  # Month=1..12, columns 2020..2025 with monthly averages


,Month,2020,2021,2022,2023,2024,2025
0,1,NaN,19.242583,14.347447,16.239041,15.458333,11.590278
1,2,NaN,23.939959,13.916550,15.411784,12.444444,11.340278
2,3,NaN,26.403541,16.278777,15.622516,14.513889,15.590278
3,4,NaN,31.467281,18.910845,17.546828,14.673611,15.215278
4,5,NaN,16.958166,18.092824,18.814211,14.270833,16.173611
5,6,NaN,18.550992,19.820379,18.077667,13.833333,13.756944
6,7,18.377238,20.098191,17.747228,18.460012,12.326389,13.368056
7,8,22.101457,22.167294,16.558391,17.650424,12.916667,13.775000
8,9,21.969967,17.796412,12.848842,17.055032,13.583333,NaN
9,10,20.764038,15.365806,16.577029,15.969356,13.944444,NaN


## Unpivot the monthly df

In [15]:
def unpivot_monthly_wide(df):
    # melt year columns long
    year_cols = [c for c in df.columns if c != 'Month']
    long = df.melt(id_vars=['Month'], value_vars=year_cols,
                   var_name="Year", value_name="Gas Transit Days Avg")
    # build a real month date (use 1st of month)
    long["Year"] = long["Year"].astype(int)
    long["Month"] = pd.to_datetime(dict(year=long["Year"],
                                       month=long['Month'],
                                       day=1))
    out = (long.dropna(subset=["Gas Transit Days Avg"])
                .sort_values("Month")
                .reset_index(drop=True)[["Month", "Gas Transit Days Avg"]])
    return out


ma_line13_df = unpivot_monthly_wide(monthly_df)  # monthly_df is your screenshot table
ma_line13_df
 

,Month,Gas Transit Days Avg
0,2020-07-01,18.377238
1,2020-08-01,22.101457
2,2020-09-01,21.969967
3,2020-10-01,20.764038
4,2020-11-01,19.276175
...,...,...
57,2025-04-01,15.215278
58,2025-05-01,16.173611
59,2025-06-01,13.756944
60,2025-07-01,13.368056


# Plot the Actual Pipeline data vs the Implied Pipeline (a range) data

In [20]:
def bounds_from_monthly_transit(ma_df, lower_L=9.784, upper_L=11.394):
    # ma_df: columns ["Month", "Gas Transit Days Avg"]
    df = ma_df[["Month", "Gas Transit Days Avg"]].copy()
    df["date"] = pd.to_datetime(df["Month"])
    df["T"]    = pd.to_numeric(df["Gas Transit Days Avg"])
    df["Q_low_mbd"]  = lower_L / df["T"]
    df["Q_high_mbd"] = upper_L / df["T"]
    return df[["date", "Q_low_mbd", "Q_high_mbd"]].sort_values("date")

def plot_bounds_vs_eia(ma_df, eia_df, lower_L=9.784, upper_L=11.394):
    # Build bounds from monthly transit averages
    b = bounds_from_monthly_transit(ma_df, lower_L, upper_L)

    # Prepare EIA P3→P1 gasoline receipts (kbd -> Mb/d)
    a = eia_df[["period", "value"]].copy()
    a["date"]   = pd.to_datetime(a["period"])
    a["actual"] = pd.to_numeric(a["value"]) / 1000.0* 0.65
    a = a[["date", "actual"]].sort_values("date")

    # Merge on date (asof handles slight mismatches)
    m = pd.merge_asof(b.sort_values("date"), a.sort_values("date"), on="date")

    # Long form for px.line
    m = m.rename(columns={
        "Q_low_mbd":  "7 / T (Mb/d)",
        "Q_high_mbd": "8.61 / T (Mb/d)",
        "actual":     "P3→P1 Gasoline (Mb/d)"
    })
    long = m.melt(id_vars="date", value_vars=["7 / T (Mb/d)", "8.61 / T (Mb/d)", "P3→P1 Gasoline (Mb/d)"],
                  var_name="Series", value_name="Mb/d")

    fig = px.line(long, x="date", y="Mb/d", color="Series",
                  title="Line 1 Bounds from Transit vs P3→P1 Gasoline Pipeline Receipts")
    fig.update_layout(xaxis_title="Month", yaxis_title="Mb/d", legend_title_text="")
    fig.show()



In [21]:
plot_bounds_vs_eia(ma_line13_df, EIA_df)